In [115]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import math
import time
from tqdm import tqdm

In [116]:
links_df = pd.read_csv("scrape_input.csv")
links_df

,City,Link
0,New York,https://www.booking.com/searchresults.html?aid...
1,Phoenix,https://www.booking.com/searchresults.html?aid...
2,Las Vegas,https://www.booking.com/searchresults.html?aid...


In [118]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

for input_df_index in range(len(links_df)):
    url = links_df["Link"][input_df_index]
    city_name = links_df["City"][input_df_index]
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'lxml')
    try:
        total_hotels_count = int(soup.select('.efdb2b543b')[0].select('.fda3b74d0d')[0].get_text().split(":")[1].split()[0])
    except:
        total_hotels_count = int(soup.select('.efdb2b543b')[0].select('.d3a14d00da')[0].get_text().split(":")[1].split()[0])  
    pages = math.ceil(total_hotels_count / 25)
    offset = 0
    time.sleep(5)
    scraped_df = pd.DataFrame(columns=('hotel_name', 'address', 'stars', 'rating', 'review_count', 'actual_price', 'discount_price', 'taxes'))
    index = 0
    print(f"Starting Scrape for {city_name}...")
    for page in tqdm(range(pages+1)):
        url = url + f"&offset={offset}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'lxml')
        hotels = soup.select('.da89aeb942')
        for item in hotels:
            hotel_name = item.select('.a23c043802')[0].get_text()
            address = item.select('.b4273d69aa')[0].get_text()
            try:
                stars = len(item.select('.fe621d6382'))
            except:
                stars = "Not Provided"
            try:
                rating = item.select('.d10a6220b4')[0].get_text()
            except:
                rating = "Not Provided"
            try:
                review_count = item.select('.db63693c62')[0].get_text()
            except:
                review_count = "Not Provided"
            try:
                actual_price = item.select('.e293a04099')[0].get_text()
            except:
                actual_price = item.select('.bd73d13072')[0].get_text()
            discount_price = item.select('.bd73d13072')[0].get_text()
            taxes = item.select('.c257e924ca')[0].get_text()

            data = [hotel_name, address, stars, rating, review_count, actual_price, discount_price, taxes]
            scraped_df.loc[index] = data
            index += 1
        offset += 25
        time.sleep(5)
    scraped_df.to_csv(f"./scraped_data/{city_name}_scraped.csv", index=False)
    print(f"Saved Scrape Data of {city_name}.")

Starting Scrape for New York...


100%|██████████| 18/18 [02:30<00:00,  8.34s/it]


Saved Scrape Data of New York.
Starting Scrape for Phoenix...


100%|██████████| 13/13 [01:47<00:00,  8.25s/it]


Saved Scrape Data of Phoenix.
Starting Scrape for Las Vegas...


100%|██████████| 10/10 [01:40<00:00, 10.01s/it]

Saved Scrape Data of Las Vegas.


In [126]:
pd.read_csv("scraped_data\Phoenix_scraped.csv")

,hotel_name,address,stars,rating,review_count,actual_price,discount_price,taxes
0,Evolve Desert Home with Mtn View - Walk to Tra...,"Ahwatukee Foothills, Phoenix",3,Not Provided,41 external reviews,"₹ 87,111","₹ 87,111","+₹ 48,000 taxes and charges"
1,SureStay Hotel by Best Western Phoenix Airport,"Downtown Phoenix, Phoenix",3,7.3,"3,620 reviews","₹ 52,123","₹ 52,123","+₹ 6,552 taxes and charges"
2,Sonder at Hance Park,"Downtown Phoenix, Phoenix",4,8.5,140 reviews,"₹ 52,123","₹ 52,123","+₹ 11,254 taxes and charges"
3,La Quinta by Wyndham Phoenix Chandler,"Ahwatukee Foothills, Phoenix",3,8.1,764 reviews,"₹ 53,366","₹ 53,366","+₹ 6,708 taxes and charges"
4,Hyatt Place Phoenix-North,"North Mountain, Phoenix",4,7.3,"1,091 reviews","₹ 39,451","₹ 39,451","+₹ 4,959 taxes and charges"
...,...,...,...,...,...,...,...,...
320,"Home2 Suites By Hilton Phoenix Airport North, Az","Camelback East, Phoenix",3,9.0,369 reviews,"₹ 63,282","₹ 63,282","+₹ 7,954 taxes and charges"
321,Holiday Inn Express & Suites - Phoenix North -...,"Desert View, Phoenix",3,8.4,279 reviews,"₹ 69,354","₹ 69,354","+₹ 8,718 taxes and charges"
322,Hilton Garden Inn Phoenix Midtown,"Encanto, Phoenix",3,7.4,259 reviews,"₹ 70,931","₹ 70,931","+₹ 8,916 taxes and charges"
323,Radisson Hotel Phoenix Airport,"Camelback East, Phoenix",3,7.8,326 reviews,"₹ 68,354","₹ 68,354","+₹ 8,592 taxes and charges"
